In [14]:
import random

from keras.layers import Activation, Dense, GRU, LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
import numpy

47+21 : 두자리수 + 두자리수 : 5자리
68

In [2]:
DIGITS = 2
MAX_LEN = 2 * DIGITS + 1
NUM_CHAR = len('0123456789+')  # 각 글자의 위치. 11.

In [3]:
N = 10000
x = numpy.zeros((N, MAX_LEN, NUM_CHAR), dtype=numpy.bool)      # 5 * 11 matrix
y = numpy.zeros((N, DIGITS + 1, NUM_CHAR), dtype=numpy.bool)   # 3 * 11 (100자리, 10자리, 1자리)

In [32]:
for i in range(1):
    a = random.randint(1, 99)
    b = random.randint(1, 99)
    
    x[i, 0, a // 10] = True
    x[i, 1, a % 10] = True
    x[i, 2, 10] = True        # '+' 위치
    x[i, 3, b // 10] = True
    x[i, 4, b % 10] = True
    
    y[i, 0, (a + b) // 100] = True           # 100 자리
    y[i, 1, (a + b) % 100 // 10] = True      # 10 자리
    y[i, 2, (a + b) % 10] = True             # 1 자리
    
    print(a, b, a + b)

41 82 123


In [33]:
x[0]

array([[False, False,  True, False,  True, False, False,  True, False,
        False, False],
       [False,  True, False, False, False,  True, False,  True,  True,
        False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True],
       [False,  True, False, False,  True, False, False, False,  True,
         True, False],
       [False, False,  True, False, False,  True,  True,  True, False,
         True, False]], dtype=bool)

In [34]:
y[0]

array([[ True,  True, False, False, False, False, False, False, False,
        False, False],
       [False, False,  True, False, False, False, False,  True, False,
         True, False],
       [ True, False, False,  True,  True, False, False,  True, False,
        False, False]], dtype=bool)

In [4]:
# 10,000 개 데이터셋 생성
for i in range(N):
    a = random.randint(1, 99)
    b = random.randint(1, 99)
    
    x[i, 0, a // 10] = True
    x[i, 1, a % 10] = True
    x[i, 2, 10] = True        # '+' 위치
    x[i, 3, b // 10] = True
    x[i, 4, b % 10] = True
    
    y[i, 0, (a + b) // 100] = True           # 100 자리
    y[i, 1, (a + b) % 100 // 10] = True      # 10 자리
    y[i, 2, (a + b) % 10] = True             # 1 자리

In [5]:
HIDDEN = 128

In [15]:
# GRU : LSTM 개선한 것.
model = Sequential()
model.add(GRU(HIDDEN, input_shape=(MAX_LEN, NUM_CHAR)))   # 5자리. 11+22. output 없음.
model.add(RepeatVector(DIGITS + 1))   # 3개로 각각 넘김.
model.add(GRU(HIDDEN, activation='relu', return_sequences=True)) # 각 노드가 모두 output
model.add(GRU(HIDDEN, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(NUM_CHAR)))                       # 각 결과를 합친다.
model.add(Activation('softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 128)               53760     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 3, 128)            0         
_________________________________________________________________
gru_2 (GRU)                  (None, 3, 128)            98688     
_________________________________________________________________
gru_3 (GRU)                  (None, 3, 128)            98688     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 11)             1419      
_________________________________________________________________
activation_2 (Activation)    (None, 3, 11)             0         
Total params: 252,555.0
Trainable params: 252,555.0
Non-trainable params: 0.0
________________________________________________________________

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x, y, batch_size=100, epochs=50, verbose=1)

Epoch 1/50
10000/10000 [==============================] - 10s - loss: 1.9359 - acc: 0.2720    
Epoch 2/50
10000/10000 [==============================] - 8s - loss: 1.6311 - acc: 0.3668     
Epoch 3/50
10000/10000 [==============================] - 8s - loss: 1.1577 - acc: 0.5590     
Epoch 4/50
10000/10000 [==============================] - 9s - loss: 0.9600 - acc: 0.6353     
Epoch 5/50
10000/10000 [==============================] - 9s - loss: 0.9155 - acc: 0.6496     
Epoch 6/50
10000/10000 [==============================] - 9s - loss: 0.8570 - acc: 0.6740     
Epoch 7/50
10000/10000 [==============================] - 9s - loss: 0.8117 - acc: 0.6900     
Epoch 8/50
10000/10000 [==============================] - 8s - loss: 0.7104 - acc: 0.7256     
Epoch 9/50
10000/10000 [==============================] - 9s - loss: 0.4556 - acc: 0.8418     
Epoch 10/50
 9200/10000 [==========================>...] - ETA: 0s - loss: 0.2489 - acc: 0.9186

In [10]:
p = model.predict(x[0:20])

In [12]:
for i in range(20):
    print('x', numpy.argmax(x[i, 0]), numpy.argmax(x[i, 1]), '+', numpy.argmax(x[i, 3]), numpy.argmax(x[i, 4]))
    print('y', numpy.argmax(y[i, 0]), numpy.argmax(y[i, 1]), numpy.argmax(y[i, 2]))
    print('p', numpy.argmax(p[i, 0]), numpy.argmax(p[i, 1]), numpy.argmax(p[i, 2]))
    print('-' * 30)

x 1 7 + 6 0
y 0 7 7
p 0 7 8
------------------------------
x 7 6 + 2 5
y 1 0 1
p 1 0 1
------------------------------
x 8 7 + 4 9
y 1 3 6
p 1 3 7
------------------------------
x 2 9 + 4 7
y 0 7 6
p 0 7 8
------------------------------
x 0 7 + 8 9
y 0 9 6
p 0 9 7
------------------------------
x 5 1 + 5 9
y 1 1 0
p 1 1 0
------------------------------
x 6 2 + 5 4
y 1 1 6
p 1 1 7
------------------------------
x 7 2 + 1 9
y 0 9 1
p 0 9 1
------------------------------
x 1 3 + 9 5
y 1 0 8
p 1 0 9
------------------------------
x 3 6 + 4 7
y 0 8 3
p 0 8 4
------------------------------
x 1 7 + 7 2
y 0 8 9
p 0 8 9
------------------------------
x 2 5 + 3 8
y 0 6 3
p 0 6 1
------------------------------
x 5 8 + 3 3
y 0 9 1
p 0 9 1
------------------------------
x 8 9 + 0 2
y 0 9 1
p 0 9 1
------------------------------
x 5 0 + 1 0
y 0 6 0
p 0 6 1
------------------------------
x 3 3 + 7 3
y 1 0 6
p 1 0 6
------------------------------
x 2 1 + 5 7
y 0 7 8
p 0 7 8
----------------------------

In [ ]:
# LSTM / GRU / MUT